In [29]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np

In [32]:
df_Entities = pd.read_csv("../Data/Entidades.csv")
df_Entities['Incidencia Semanal Normalizada'] = np.random.random(33)
df_Entities['Tasa de Cambio'] = np.random.random(33)-.5

In [38]:
fig1 = px.bar(df_Entities, x='Entidad', y='Incidencia Semanal Normalizada')
fig2 = px.bar(df_Entities, x='Entidad', y='Tasa de Cambio')

fig = make_subplots(rows=2, cols=1)
fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=2, col=1)
fig.update_layout(title_text="Útlima actialización: "+pd.to_datetime('today') .strftime('%d-%m-%y'),
    height=600
    )
fig.update_xaxes(showticklabels=False, title=None, row=1, col=1)
fig.update_yaxes(title="Incidencia Semanal Normalizada", row=1, col=1)
fig.update_yaxes(title="Tasa de Cambio", row=2, col=1)

fig.show()
fig.write_html("../docs/Fig01.html")